In [1]:
from google.colab import files
uploaded = files.upload()



Saving discretized_data_bin.xlsx to discretized_data_bin.xlsx


In [2]:
pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.14.0
    Uninstalling tensorflow-2.14.0:
      Successfully

In [3]:
pip install --upgrade stable-baselines3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 12.8 MB/s eta 0:00:00


In [11]:
!pip install stable-baselines3 gymnasium shimmy






In [12]:
# Caricare il dataset
import pandas as pd

df = pd.read_excel('discretized_data_bin.xlsx')

In [13]:
df

,KEY,SKU_CODE,3N_PRICE_LIST_bin,Purch_qnt_bin,Stock_Ave_QNT_bin,Lead_Time_Ave_DAYS_bin,Sales_QTY_External_bin,Sales_TO_External_bin,FP_EUR_External_sales_bin
0,K.11108494,A.11108494,0,9,9,9,9,8,8
1,K.11110176,A.11110176,1,9,9,9,9,9,9
2,K.11110175,A.11110175,1,9,9,8,9,9,9
3,K.11103933,A.11103933,0,9,9,9,9,8,8
4,K.11110175,B.11110175,0,9,9,7,9,7,7
...,...,...,...,...,...,...,...,...,...
1801,K.11109560,B.11109560,7,0,0,5,0,1,1
1802,K.11109588,B.11109588,3,0,0,4,0,0,0
1803,K.11109703,B.11109703,6,0,3,1,3,4,4
1804,K.11110611,B.11110611,0,0,0,6,0,0,0


In [42]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import random




In [46]:


class CustomEnvironment(gym.Env):
    mix_ratios = {
        0: (0.80, 0.20),  # 80% A e 20% B
        1: (0.65, 0.35),  # 65% A e 35% B
        2: (0.50, 0.50),  # 50% A e 50% B
        3: (0.35, 0.65),  # 35% A e 65% B
        4: (0.20, 0.80)   # 20% A e 80% B
    }

    def __init__(self, df):
        super(CustomEnvironment, self).__init__()
        self.action_space = spaces.Discrete(5)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(df.shape[1],), dtype=np.float64)
        self.df = df
        self.current_step = 0
        self.a_b_pairs = self.extract_a_b_pairs()

    def extract_a_b_pairs(self):
        a_b_pairs = {}
        for key in self.df['KEY'].unique():
            a_values = self.df[self.df['KEY'] == key]['SKU_CODE'].tolist()
            b_values = self.df[self.df['KEY'] == key]['SKU_CODE'].tolist()
            a_b_pairs[key] = (a_values, b_values)
        return a_b_pairs

    def _get_observation(self):
        return self.df.iloc[self.current_step]

    def step(self, action):
        key = self.df.iloc[self.current_step]['KEY']
        a_b_pair = self.a_b_pairs[key]
        action_result = self.perform_action(a_b_pair, action)

        # Continua ciclicamente attraverso i dati
        self.current_step = (self.current_step + 1) % len(self.df)
        done = False  # Imposta done a False per continuare ciclicamente

        self.state = self._get_observation()
        reward = self.calculate_reward(self.current_step)
        return self.state, reward, done, {}

    def perform_action(self, a_b_pair, action):
        a_ratio, b_ratio = self.mix_ratios[action]
        return a_ratio, b_ratio

    def calculate_reward(self, state_index):
        bin_labels = {0: -3, 1: -2, 2: -1, 3: -0.5, 4: 0, 5: 0, 6: 0.5, 7: 1, 8: 2, 9: 3}
        state_row = self.df.iloc[state_index]
        sales_bin = state_row['FP_EUR_External_sales_bin']
        return bin_labels.get(sales_bin, 0)

    def reset(self):
        self.current_step = 0
        self.state = self._get_observation()
        return self.state

    def render(self, mode='human', close=False):
        print(f'Stato: {self.state}')

    def shuffle_data(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)





In [47]:
# Carica il dataset
df = pd.read_excel('discretized_data_bin.xlsx')
env = CustomEnvironment(df)
env.shuffle_data()



In [48]:
class QLearningAgent:
    def __init__(self, n_actions, learning_rate=0.01, discount_factor=0.99, exploration_rate=1.0, exploration_decay_rate=0.001):
        self.n_actions = n_actions
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = exploration_rate
        self.epsilon_decay = exploration_decay_rate
        self.q_table = {}

    def map_state(self, state):
        state_key = tuple(state)
        if state_key not in self.q_table:
            self.q_table[state_key] = np.zeros(self.n_actions)
        return state_key

    def choose_action(self, state):
        state_key = self.map_state(state)
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.n_actions)
        else:
            return np.argmax(self.q_table[state_key])

    def learn(self, state, action, reward, next_state):
        state_key = self.map_state(state)
        next_state_key = self.map_state(next_state)
        predict = self.q_table[state_key][action]
        target = reward + self.gamma * np.max(self.q_table[next_state_key])
        self.q_table[state_key][action] += self.lr * (target - predict)
        self.epsilon = max(self.epsilon - self.epsilon_decay, 0.01)

# Il resto del tuo codice di inizializzazione e ciclo di apprendimento rimane invariato


In [49]:
# Numero di azioni
n_actions = env.action_space.n

# Creazione dell'agente Q-Learning
agent = QLearningAgent(n_actions)

# Ciclo di apprendimento
n_episodes = 1000
for episode in range(n_episodes):
    state = env.reset()
    done = False

    while not done:
        # Validazione dell'azione
        action = agent.choose_action(state)
        if action not in range(n_actions):
            # Gestire l'azione invalida
            action = env.sample_action()

        next_state, reward, done, _ = env.step(action)
        agent.learn(state, action, reward, next_state)
        state = next_state

    env.shuffle_data()

    if episode % 100 == 0:
        print(f"Episodio: {episode}, Esplorazione: {agent.epsilon:.2f}")


KeyboardInterrupt: ignored